In [1]:
# Instalar otras dependencias para ayudarnos
#!pip install tensorflow
#!pip install -q "tqdm>=4.36.1"
# !pip install tensorflow-addons==0.22.0


In [2]:
# importar librerías
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
import os
import tqdm
# import tensorflow_addons as tfa
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

2.16.1
[]


In [12]:
# Cargamos la data
data = pd.read_csv("data.csv", header=0)
# Hacemos un shuffle de los rows
data = data.sample(frac = 1)
data.head(10)

,pelvic_incidence,pelvic_tilt numeric,lumbar_lordosis_angle,sacral_slope,pelvic_radius,degree_spondylolisthesis,class
209,48.259920,16.417462,36.329137,31.842457,94.882336,28.343799,Abnormal
198,74.854480,13.909084,62.693259,60.945396,115.208701,33.172255,Abnormal
10,49.706610,13.040974,31.334500,36.665635,108.648265,-7.825986,Abnormal
38,55.843286,28.847448,47.690543,26.995838,123.311845,2.812427,Abnormal
46,48.332638,22.227784,36.181993,26.104854,117.384625,6.481709,Abnormal
238,69.004913,13.291790,55.570143,55.713123,126.611621,10.832011,Normal
93,58.101935,14.837639,79.649838,43.264295,113.587655,50.237878,Abnormal
51,74.433593,41.557331,27.700000,32.876262,107.949304,5.000089,Abnormal
17,31.276012,3.144669,32.562996,28.131342,129.011418,3.623020,Abnormal
88,56.605771,16.800200,42.000000,39.805571,127.294522,24.018575,Abnormal


In [13]:
#Creamos una lista de las posibles clases y su representación inversa
class_names = {
    "Abnormal": 0,
    "Normal": 1
}
reversed_class_names = {v: k for k, v in class_names.items()}
data['class'] = data['class'].map(class_names)
data.head(10)



,pelvic_incidence,pelvic_tilt numeric,lumbar_lordosis_angle,sacral_slope,pelvic_radius,degree_spondylolisthesis,class
209,48.259920,16.417462,36.329137,31.842457,94.882336,28.343799,0
198,74.854480,13.909084,62.693259,60.945396,115.208701,33.172255,0
10,49.706610,13.040974,31.334500,36.665635,108.648265,-7.825986,0
38,55.843286,28.847448,47.690543,26.995838,123.311845,2.812427,0
46,48.332638,22.227784,36.181993,26.104854,117.384625,6.481709,0
238,69.004913,13.291790,55.570143,55.713123,126.611621,10.832011,1
93,58.101935,14.837639,79.649838,43.264295,113.587655,50.237878,0
51,74.433593,41.557331,27.700000,32.876262,107.949304,5.000089,0
17,31.276012,3.144669,32.562996,28.131342,129.011418,3.623020,0
88,56.605771,16.800200,42.000000,39.805571,127.294522,24.018575,0


In [14]:
#Definimos los porcentajes de entrenamiento:

dataset_percent_for_training = 0.8
dataset_percent_for_testing = 0.2
data_count = len(data)
training_count = int((data_count * dataset_percent_for_training))
testing_count = int(data_count * dataset_percent_for_testing)

# Creamos los sets de entrenamiento y de pruebas
training_data = data.iloc[:, :5][:training_count]
training_labels = data.iloc[:, 6][:training_count]

testing_data = data.iloc[:, :5][training_count:training_count + testing_count]
testing_labels = data.iloc[:, 6][training_count:training_count + testing_count]
print(testing_labels.head(10))

210    1
62     0
249    1
55     0
196    0
287    1
296    1
197    0
301    1
83     0
Name: class, dtype: int64


In [5]:
# Creamos una red neuronal. Esta red neuronal va a tener 6 neuronas iniciales,
# para poder ser excitadas por las señales de las columnas en la data.
# Tendrá una sola capa ya que las 6 neuronas tienen un valor "relevante" para el final.
# Habrá una sola neurona final que clasifique el resultado como Normal (1) o Anormal (0)
test_acc = 0
train_model = False
model = None
while train_model and test_acc < 0.9:
    model = keras.Sequential([
        keras.layers.Dense(6, activation='softmax'),
        keras.layers.Dense(2)
    ])

    optimizer = tf.keras.optimizers.Adam(
        learning_rate=0.0007
    )
    model.compile(optimizer=optimizer,
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                    metrics=['accuracy'])
    #Definimos la cantidad de epocs de entrenamiento.
    epochs = 150


    #Comenzamos el entrenamiento
    model.fit(training_data, training_labels, epochs=epochs, verbose=0)                         
    test_loss, test_acc = model.evaluate(testing_data, testing_labels, verbose=2)

In [6]:
def load_saved_model(model_name):
    model_path = f"models/{model_name}"
    model = tf.keras.models.load_model(model_path)
    print(f'Model loaded from {model_path}')
    return model

# Si no hay modelo creado, preguntemos si queremos cargar uno

model_name = 'trained_model-08.keras'
model = load_saved_model(model_name)
test_loss, test_acc = model.evaluate(testing_data, testing_labels, verbose=2)
print(model)
    

Model loaded from models/trained_model-08.keras


NameError: name 'testing_data' is not defined

In [3]:
#Evaluemos el modelo
print('\nTest accuracy: ', test_acc)

NameError: name 'test_acc' is not defined

In [ ]:
#Guardemos el modelo

# model_path = 'models/trained_model-09.keras'
# model.save(model_path)

In [ ]:
#Hagamos predicciones
probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])
predictions = probability_model.predict(testing_data)

slice = 10
end = slice + 30
selected_predictions = predictions[slice:end]
selected_actual = testing_labels[slice:end]

fig, axs = plt.subplots(5, 4, figsize=(15, 20))

for i, ax in enumerate(axs.flat):
    # Plotear el valor predicho
    ax.bar(reversed_class_names.values(), selected_predictions[i], alpha= 0.5, label='Predicho')
    # Plotear el valor real
    actual_values = [0] * len(reversed_class_names)
    actual_values[selected_actual.iloc[i]] = 1
    ax.bar(reversed_class_names.values(), actual_values, alpha=0.5, label='Real')
    ax.set_title(f'Predicción {i + 1}')
    ax.set_ylabel('Confianza')
    ax.set_ylim([0, 1])
    ax.legend()
plt.tight_layout()
plt.show()